In [1]:
import yfinance as yf # For loading financial datasets
import pandas as pd # For creating tables and dataframes
import math 
import numpy as np

stocks = ["AAPL", "AMZN", "MSFT", "NFLX", "GOOG", "TSLA", "GME"] # 7 stocks needed for the first table

In [2]:
weight = 1 / 7 # This will be the portfolio weight for each ticker 
portfolio_weight = [weight]*7

In [29]:
def get_annualized_volatility(stocks):
    dataframe = yf.download(f"{stocks}")["Adj Close"]
    dataframe_rets = dataframe.pct_change() #Calculate returns for every 3-months
    dataframe.dropna(inplace=True) #Drop any NaN values (Must drop!)
    annualized_volatility = dataframe_rets.std() * math.sqrt(4)
    return annualized_volatility

In [38]:
get_annualized_volatility("AMZN")

[*********************100%***********************]  1 of 1 completed


0.07225227145995164

In [31]:
def beta_against_market(ticker, market):
    dataframe = yf.download(f"{ticker} {market}")["Adj Close"]
    dataframe.dropna(inplace=True)
    dataframe_resampled = dataframe.resample("12M").sum() # Resample data to use trailing 12-months
    covariance = dataframe_resampled.cov().iloc[0, 1] # left of the comma is for row selection [row, column]
    beta = covariance / dataframe_resampled[market].var()
    return beta

In [57]:
def get_weekly_drawdowns(ticker):
    dataframe = yf.download(f"{ticker}")
    weekly_drawdowns = []
    weekly_resample = dataframe.resample("5D").asfreq()["Adj Close"] #resample for weekly data (5 trading days per week)
    weekly_resample.dropna(inplace=True)
    weekly_resample_array = np.array(weekly_resample)
    for i in range(0, len(weekly_resample_array)-52, 52): # Breakdown dataset into 52 weeks to find max and minimum during the 52-week period
        high = np.max(weekly_resample_array[i:i+52])
        low = np.min(weekly_resample_array[i:i+52])
        calc = (low - high) / high
        weekly_drawdowns.append(calc)
    average_weekly_drawdowns = sum(weekly_drawdowns) / len(weekly_drawdowns)
    maximum_weekly_drawdown = max(weekly_drawdowns)
    return average_weekly_drawdowns, maximum_weekly_drawdown

In [58]:
def get_total_return(ticker):
    dataframe = yf.download(f"{ticker}")["Adj Close"]
    #.iloc[-1] get last row and .iloc[0] get first row of Adj Close 
    #Formula for Total_Return = New_Stock_price - Current_Stock_Price / Current_Stock_Price
    total_return = (dataframe.iloc[-1] - dataframe.iloc[0]) / (dataframe.iloc[0])
    return f"{total_return*100}%"

In [61]:
def get_total_annualized_return(stock):
    dataframe = yf.download(f"{stock}")["Adj Close"]
    resampled_data = dataframe.resample("10Y").asfreq()
    resampled_data.dropna(inplace=True)
    total_annualized_return = (resampled_data.iloc[-1] - resampled_data.iloc[0]) / resampled_data.iloc[0]
    return f"{total_annualized_return * 100}%"

In [62]:
stocks = ["AAPL", "AMZN", "MSFT", "NFLX", "GOOG", "TSLA", "GME"] # 7 stocks needed for the first table


#Create table
tickers = []
annualized_volatility = []
beta_spy = []
beta_iwm = []
beta_dia = []
average_weekly_drawdown = []
maximum_weekly_drawdown = []
total_return = []
total_annualized_return = []
for stock in stocks: # This will loop through the 7 stocks to get the data needed
    get_dataframe = yf.download(f"{stock}")
    get_dataframe.dropna(inplace=True) #Drop any NaN values (Must drop!)
    tickers.append(stock)
    annualized_volatility.append(get_annualized_volatility(stock))
    beta_spy.append(beta_against_market(stock, "SPY"))
    beta_iwm.append(beta_against_market(stock, "IWM"))
    beta_dia.append(beta_against_market(stock, "DIA"))
    average_weekly_drawdown.append(get_weekly_drawdowns(stock)[0])
    maximum_weekly_drawdown.append(get_weekly_drawdowns(stock)[1])
    total_return.append(get_total_return(stock))
    total_annualized_return.append(get_total_annualized_return(stock))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%********

In [63]:
create_table = {"ticker" : tickers, "portfolio weight": portfolio_weight, 
                "annualized volatility" : annualized_volatility, "beta spy" : beta_spy,
               "beta iwm": beta_iwm, "beta dia": beta_dia, "average weekly drawdown": average_weekly_drawdown,
               "maximum weekly drawdown": maximum_weekly_drawdown, "total return" : total_return, 
               "total annualized return" : total_annualized_return}
create_dataframe = pd.DataFrame(create_table)

In [84]:
table_one = create_dataframe
table_one

,ticker,portfolio weight,annualized volatility,beta spy,beta iwm,beta dia,average weekly drawdown,maximum weekly drawdown,total return,total annualized return
0,AAPL,0.142857,0.056478,0.346325,0.712702,0.433881,-0.463143,-0.222692,150790.1872531127%,110503.25095947844%
1,AMZN,0.142857,0.072252,0.438472,0.852993,0.496299,-0.485474,-0.252311,94296.27666515727%,1745.1095093625365%
2,MSFT,0.142857,0.042650,0.633366,1.283657,0.780965,-0.354988,-0.131717,418518.5013779911%,6064.755168210947%
3,NFLX,0.142857,0.071776,1.413053,2.833438,1.743365,-0.543612,-0.283618,24432.17084178011%,1581.9246861227023%
4,GOOG,0.142857,0.038881,0.311764,0.616832,0.379562,-0.351752,-0.155658,3567.6719336653528%,446.6324740288087%
5,TSLA,0.142857,0.072330,0.644040,1.191835,0.757325,-0.514386,-0.288251,10855.20893719201%,13149.53242129483%
6,GME,0.142857,0.101284,0.063133,0.120999,0.073073,-0.501950,-0.274563,890.1477506899047%,431.3860768298257%


In [85]:
#Prepare annual data for sharpe ratio
get_stock_data = yf.download(stocks)["Adj Close"]
get_stock_data.dropna(inplace=True)
data_resampled = get_stock_data.resample("Y").asfreq()
data_resampled.dropna(inplace=True)


annual_pct_change = data_resampled.pct_change()[1:] 
weighted_annual_rets = annual_pct_change * portfolio_weight
portfolio_annual_rets = weighted_annual_rets.sum(axis=1)

#Compute Sharpe Ratio
expected_portfolio_return = portfolio_annual_rets.mean()
portfolio_std = portfolio_annual_rets.std()
risk_free_rate = 0.0471 # 3-Months T-bill current interest


sharpe_ratio = (expected_portfolio_return - risk_free_rate) / portfolio_std

[*********************100%***********************]  7 of 7 completed


1.1070720013189557

In [86]:
#Annualized Volatility spread (Portfilio Volatility - ETF Volatility)
dataframe = yf.download(stocks)["Adj Close"]
dataframe.dropna(inplace=True)
dataframe_resampled = dataframe.resample("252D").asfreq() # Resample every 252 days
dataframe_resampled.dropna(inplace=True)

[*********************100%***********************]  7 of 7 completed


In [87]:
dataframe_pct_change = dataframe_resampled.pct_change()[1:]

In [88]:
portfolio_ret_252d = dataframe_pct_change * portfolio_weight
final_return_252d = portfolio_ret_252d.sum(axis=1)

In [89]:
final_return_252d.std()

0.5831557666671989

In [90]:
etf_volatility = yf.download("SPY")["Adj Close"]
resample_etf_252 = etf_volatility.resample("252D").asfreq()
resample_etf_252.pct_change()[1:].std()


[*********************100%***********************]  1 of 1 completed


0.13048282413266848

In [91]:
def get_corr_against_etf(data):
    return data.corr().iloc[0, 1]

def get_cov_against_etf(data):
    return data.corr().iloc[0, 1]

def get_tracking_error(data):
    data["P - B"] = data["portfolio_rets"] - data["daily_etf_rets"]
    return data["P - B"].std()

def get_volatility_spread(stock_data, etf):
    stock_data_resampled = stock_data.resample("252D").asfreq()
    stock_data_resampled.dropna(inplace=True)
    stock_data_pct = stock_data_resampled.pct_change()[1:]
    portfolio_ret_252d = stock_data_pct * portfolio_weight
    final_return_252d = portfolio_ret_252d.sum(axis=1)
    portfolio_volatility = final_return_252d.std()
    get_etf = yf.download(etf)["Adj Close"].pct_change()[1:]
    etf_volatility = get_etf.std()
    return portfolio_volatility - etf_volatility

In [92]:
etf_tickers = ["SPY", "IWM", "DIA"]

correlations = []
covariances = []
sharpe_ratios = [sharpe_ratio]*3
tracking_errors = []
volatility_spreads = []


for ticker in etf_tickers:
    # To get correlation of portfolio against ETF we first need to get daily returns of portfolio
    stock_data = yf.download(stocks)["Adj Close"]
    stock_data.dropna(inplace=True)
    
    weighted_rets = (portfolio_weight * stock_data) #stock * weight = portfolio_return accross all 7 stocks 
    portfolio_rets = weighted_rets.sum(axis=1) #https://www.codingfinance.com/post/2018-04-05-portfolio-returns-py/
    
    
    etf = yf.download(ticker, start="2010-06-29")["Adj Close"]
    etf_pct_change = etf.pct_change()[1:]
    portfolio_rets_df = pd.DataFrame(portfolio_rets) #convert type series to dataframe
    portfolio_rets_df["daily_etf_rets"] = etf_pct_change
    portfolio_rets_df.columns = ["portfolio_rets", "daily_etf_rets"]
    correlations.append(get_corr_against_etf(portfolio_rets_df))
    covariances.append(get_cov_against_etf(portfolio_rets_df))
    tracking_errors.append(get_tracking_error(portfolio_rets_df))
    volatility_spreads.append(get_volatility_spread(stock_data, ticker))

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [93]:
table_two = {"tickers": etf_tickers, "correlation" : correlations, "covariances": covariances, 
             "sharpe_ratio": sharpe_ratios, "tracking error": tracking_errors, 
             "annualized_volatility_spread": volatility_spreads}
table_two = pd.DataFrame(table_two)

In [94]:
table_two

,tickers,correlation,covariances,sharpe_ratio,tracking error,annualized_volatility_spread
0,SPY,0.002193,0.002193,1.107072,69.373516,0.571240
1,IWM,-0.003277,-0.003277,1.107072,69.373587,0.567864
2,DIA,-0.000771,-0.000771,1.107072,69.373549,0.571115


1.1070720013189557